In [3]:
import gym
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Load dataset
df = pd.read_csv("data\something_new.csv")
# df = df[df["Year"] == 2023]  # Filter for Year 2023

# Define Action Space: 0 -> Buy, 1 -> Sell, 2 -> Use
ACTIONS = ["Buy", "Sell", "Use"]

# Define Environment
class FleetEnv(gym.Env):
    def __init__(self, data):
        super(FleetEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.state_size = 4  # Num_Vehicles, Cost, Emissions, Fuel Cost
        self.action_space = gym.spaces.Discrete(len(ACTIONS))  
        self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape=(self.state_size,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        return self.get_state()

    def get_state(self):
        row = self.data.iloc[self.current_step]
        return np.array([row["Num_Vehicles"], row["Cost ($)"], row["carbon_er"], row["fuel_costs"]], dtype=np.float32)
    
    def step(self, action):
        row = self.data.iloc[self.current_step]
        reward = - (row["carbon_er"] + row["Cost ($)"])  # Minimize cost & emissions
        done = self.current_step >= len(self.data) - 1
        self.current_step += 1
        return self.get_state(), reward, done, {}

# Define DQN Network
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Training Loop
def train_dqn(env, episodes=1000):
    model = DQN(env.state_size, env.action_space.n)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    memory = deque(maxlen=10000)
    gamma = 0.95  # Discount factor
    epsilon = 1.0  # Exploration rate
    epsilon_decay = 0.995
    epsilon_min = 0.01

    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            if random.random() < epsilon:
                action = env.action_space.sample()  # Explore
            else:
                with torch.no_grad():
                    action = torch.argmax(model(torch.tensor(state, dtype=torch.float32))).item()  # Exploit

            next_state, reward, done, _ = env.step(action)
            memory.append((state, action, reward, next_state, done))
            state = next_state
            total_reward += reward

            if len(memory) > 32:  # Train after 32 experiences
                batch = random.sample(memory, 32)
                states, actions, rewards, next_states, dones = zip(*batch)
                states = torch.tensor(states, dtype=torch.float32)
                actions = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)
                rewards = torch.tensor(rewards, dtype=torch.float32)
                next_states = torch.tensor(next_states, dtype=torch.float32)
                dones = torch.tensor(dones, dtype=torch.float32)

                q_values = model(states).gather(1, actions).squeeze()
                next_q_values = model(next_states).max(1)[0].detach()
                targets = rewards + gamma * next_q_values * (1 - dones)

                loss = loss_fn(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        epsilon = max(epsilon * epsilon_decay, epsilon_min)  # Decay epsilon

        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

    return model

# Run training
env = FleetEnv(df)
dqn_model = train_dqn(env)


KeyError: 'Num_Vehicles'

In [2]:
pip install gym

     -------------------------------------- 721.7/721.7 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827704 sha256=8bc7ebfe1975b0376e27d254eee35125cada3246a4f6605a356cd1734e2cd7ee
  Stored in directory: c:\users\hjain\appdata\local\pip\cache\wheels\ae\5f\67\64914473eb34e9ba89dbc7eefe7e9be8f6673fbc6f0273b29f
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import DQN

# Load CSV data
df = pd.read_csv("data\something_new.csv")  # Replace with your actual file path
# df = df[df['Year'] == 2023]  # Filter for 2023 only

# Define vehicle types
vehicle_types = df['Fuel'].unique()
costs = df.groupby('Fuel')['Cost ($)'].mean().to_dict()
emissions = df.groupby('Fuel')['carbon_emissions'].mean().to_dict()

# Constraints
MAX_EMISSIONS = 11677957  # Given limit
NUM_VEHICLE_TYPES = len(vehicle_types)

class FleetEnv(gym.Env):
    def __init__(self):
        super(FleetEnv, self).__init__()
        
        # Action space: Number of vehicles per type
        self.action_space = spaces.MultiDiscrete([50] * NUM_VEHICLE_TYPES)  # Up to 50 per type
        
        # Observation space: Current fleet composition
        self.observation_space = spaces.Box(low=0, high=50, shape=(NUM_VEHICLE_TYPES,), dtype=np.int32)
        
        self.state = np.zeros(NUM_VEHICLE_TYPES)
        
    def step(self, action):
        self.state = action
        total_emission = sum(self.state[i] * emissions[vehicle_types[i]] for i in range(NUM_VEHICLE_TYPES))
        total_cost = sum(self.state[i] * costs[vehicle_types[i]] for i in range(NUM_VEHICLE_TYPES))
        
        # Reward function (penalizes high cost and excess emissions)
        reward = -total_cost / 1e6  # Normalize cost penalty
        if total_emission > MAX_EMISSIONS:
            reward -= 10  # Heavy penalty for exceeding emissions
        
        done = total_emission <= MAX_EMISSIONS
        return self.state, reward, done, {}
    
    def reset(self):
        self.state = np.zeros(NUM_VEHICLE_TYPES)
        return self.state

# Initialize environment
env = FleetEnv()

# Train RL model
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Find optimal fleet
done = False
obs = env.reset()
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _ = env.step(action)

print("Optimal Fleet Composition:")
for i, vehicle in enumerate(vehicle_types):
    print(f"{vehicle}: {obs[i]} vehicles")

print(f"Total Cost: ${sum(obs[i] * costs[vehicle_types[i]] for i in range(NUM_VEHICLE_TYPES)):.2f}")
print(f"Total Emissions: {sum(obs[i] * emissions[vehicle_types[i]] for i in range(NUM_VEHICLE_TYPES)):.2f}")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\hjain\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.discrete.Discrete'>,) as action spaces but MultiDiscrete([50 50 50 50 50]) was provided

In [2]:
pip install stable_baselines3

  Using cached stable_baselines3-2.5.0-py3-none-any.whl (183 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install "shimmy>=2.0"


Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
from sklearn.preprocessing import LabelEncoder

class FleetEnv(gym.Env):
    def __init__(self, csv_path):
        super(FleetEnv, self).__init__()
        self.df = pd.read_csv(csv_path)
        
        # Encode categorical columns
        self.label_encoders = {}
        for col in ['ID', 'Fuel']:
            le = LabelEncoder()
            self.df[col] = le.fit_transform(self.df[col])
            self.label_encoders[col] = le
        
        # Define state space (normalized numerical features)
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(len(self.df.columns) - 1,), dtype=np.float32
        )
        
        # Define action space: 3 discrete actions (Buy=0, Use=1, Sell=2)
        self.action_space = spaces.Discrete(3)
        
        # Internal state tracking
        self.current_index = 0
        
    def reset(self):
        self.current_index = 0
        return self._get_observation()
    
    def _get_observation(self):
        obs = self.df.iloc[self.current_index].drop(['Type']).values
        obs = obs.astype(np.float32)  # Ensure numeric format
        return obs
    
    def step(self, action):
        reward = self._calculate_reward(action)
        
        self.current_index += 1
        done = self.current_index >= len(self.df)
        
        return (self._get_observation() if not done else np.zeros_like(self._get_observation())), reward, done, {}
    
    def _calculate_reward(self, action):
        row = self.df.iloc[self.current_index]
        actual_action = {'Buy': 0, 'Use': 1, 'Sell': 2}[row['Type']]
        return 1 if action == actual_action else -1


In [12]:
env = FleetEnv("data/something_new.csv")
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\hjain\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


ValueError: could not convert string to float: 'BEV_S1_2023'

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger

# ====================
# 1. Demand Forecasting
# ====================

# Assume demand.csv has columns: Year, Size, Distance, Demand (in kms)
demand_df = pd.read_csv("data/demand.csv")

# Forecast demand per (Size, Distance) combination using Prophet.
# We create forecasts for each unique combination.
forecast_horizon = list(range(2023, 2039))
forecast_demand = {}  # dict with key: (Size, Distance) -> forecast DataFrame

for (size, bucket), group in demand_df.groupby(["Size", "Distance"]):
    # Prophet expects columns 'ds' (as datetime) and 'y' (demand value)
    df_prophet = group[['Year', 'Demand']].rename(columns={'Year':'ds', 'Demand':'y'})
    # Convert year to datetime (using Jan 1 as date)
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'].astype(str) + '-01-01')
    model = Prophet(yearly_seasonality=False, daily_seasonality=False)
    model.fit(df_prophet)
    
    # Create dataframe for future predictions from 2023 to 2038
    future = pd.DataFrame({'ds': pd.to_datetime([str(year) + '-01-01' for year in forecast_horizon])})
    forecast = model.predict(future)
    forecast_demand[(size, bucket)] = forecast[['ds', 'yhat']].rename(columns={'ds':'Year'})
    forecast_demand[(size, bucket)]['Year'] = forecast_demand[(size, bucket)]['Year'].dt.year

# ====================
# 2. Fleet Optimization Model Setup
# ====================

# Load other datasets
vehicles_df = pd.read_csv("data/vehicles.csv")  # columns: ID, Vehicle, Size, Year, Cost, Yearly_range, Distance
fuels_df = pd.read_csv("data/fuels.csv")        # columns: Fuel, Year, Emissions, Cost, Cost_Uncertainty
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")  # columns: Year, Carbon emission CO2/kg

# For simplicity, assume vehicle_fuel consumption and insurance/maintenance/resale details are pre-processed into dictionaries.
# e.g. fuel_consumption = { vehicle_ID: { fuel: consumption_value } }
# and cost_factors = { 'insurance': {year_offset: percentage}, ... }
# (You must create these from your provided tables)

# Define indices and sets
years = list(range(2023, 2039))
vehicles = vehicles_df['ID'].unique()
# Assume we have a mapping from vehicle ID to its properties:
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Decision variables:
# Let buy[year][vehicle] be the number of vehicles bought in that year of that model.
# Similarly, use[year][vehicle] be number of vehicles used, and sell[year][vehicle] be vehicles sold at end of year.
buy = {(yr, vid): LpVariable(f"buy_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles if vehicle_props[vid]['Year'] == yr}
use = {(yr, vid): LpVariable(f"use_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles}  # usage decision (should not exceed fleet available)
sell = {(yr, vid): LpVariable(f"sell_{yr}_{vid}", lowBound=0, cat=LpInteger)
        for yr in years for vid in vehicles}

# For assigning vehicles to meet demand, define an allocation variable:
# alloc[year][(size, demand_bucket)][vehicle] = number of vehicles of a given model used to satisfy demand in that bucket.
alloc = {(yr, s, d, vid): LpVariable(f"alloc_{yr}_{s}_{d}_{vid}", lowBound=0, cat=LpInteger)
         for yr in years
         for (s, d) in forecast_demand.keys()
         for vid in vehicles if vehicle_props[vid]['Size'] == s}

# Create the LP problem:
prob = LpProblem("Fleet_Optimization", LpMinimize)

# ===============
# 3. Objective Function
# ===============
# The total cost includes purchase, insurance, maintenance, fuel and subtracting resale.
# Here we only include purchase cost as an example. You should add the others similarly.
purchase_cost = lpSum(buy[(yr, vid)] * vehicle_props[vid]['Cost'] for (yr, vid) in buy)
# For resale, assume a simple factor based on vehicle age – placeholder.
resale_value = lpSum(sell[(yr, vid)] * 0.3 * vehicle_props[vid]['Cost']
                     for (yr, vid) in sell)
# Define a placeholder fuel cost term (to be replaced by actual calculations based on fuel consumption, distance allocated, and fuel cost):
fuel_cost = lpSum(alloc[(yr, s, d, vid)] * 0.5   # placeholder per km cost
                  for (yr, s, d, vid) in alloc)

prob += (purchase_cost + fuel_cost - resale_value), "Total_Fleet_Cost"

# ===============
# 4. Constraints
# ===============

# Constraint 1: Demand satisfaction – For every year and (size, distance) bucket,
# the sum of distance covered by vehicles allocated must exceed the forecasted demand.
for (s, d), df_forecast in forecast_demand.items():
    for yr in years:
        # get forecasted demand for this combination and year (if available)
        demand_val = df_forecast.loc[df_forecast['Year'] == yr, 'yhat']
        if not demand_val.empty:
            # Each vehicle has a maximum daily distance given in vehicles.csv under "Distance" mapping.
            # We assume a conversion: for a vehicle with distance bucket D, the maximum distance is defined in vehicle_props.
            prob += (lpSum(alloc[(yr, s, d, vid)] * vehicle_props[vid]['Yearly_range']
                           for vid in vehicles if vehicle_props[vid]['Size'] == s)
                     >= float(demand_val), f"Demand_{yr}_{s}_{d}")

# Constraint 2: A vehicle’s distance bucket – vehicles can only satisfy demands for buckets less than or equal to their own.
for yr in years:
    for (s, d), _ in forecast_demand.items():
        for vid in vehicles:
            if vehicle_props[vid]['Size'] == s:
                # Let vehicle_distance = vehicle_props[vid]['Distance'] (like D3)
                # If vehicle's bucket < current demand bucket d, then allocation must be zero.
                # (Here we use a simple mapping of bucket order; you must implement the actual logic.)
                vehicle_bucket = vehicle_props[vid]['Distance']
                bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
                if bucket_order[vehicle_bucket] < bucket_order[d]:
                    prob += (alloc[(yr, s, d, vid)] == 0, f"Distance_{yr}_{s}_{d}_{vid}")

# Constraint 3: Fleet availability and lifetime
# For each year and vehicle, the number used must not exceed those bought minus those sold, taking into account lifetime limits.
fleet = {}
for vid in vehicles:
    for yr in years:
        # Sum over all years of purchase that are still active
        active_fleet = lpSum(buy[(y, vid)] for y in years if y <= yr and yr - y < 10) - \
                       lpSum(sell[(y, vid)] for y in years if y < yr)
        fleet[(yr, vid)] = active_fleet
        prob += (use[(yr, vid)] <= active_fleet, f"Fleet_use_{yr}_{vid}")

# Constraint 4: Selling limits – at most 20% of the fleet can be sold in any year.
for yr in years:
    total_fleet = lpSum(fleet[(yr, vid)] for vid in vehicles)
    total_sell = lpSum(sell[(yr, vid)] for vid in vehicles)
    prob += (total_sell <= 0.20 * total_fleet, f"Sell_limit_{yr}")

# Constraint 5: A vehicle model can only be bought in its designated year.
for (yr, vid) in buy:
    model_year = vehicle_props[vid]['Year']
    if yr != model_year:
        prob += (buy[(yr, vid)] == 0, f"Purchase_year_{yr}_{vid}")

# Constraint 6: Linking allocation with usage – you cannot allocate more vehicles than available.
for yr in years:
    for vid in vehicles:
        # Sum of allocations of vehicle vid across all demand buckets for its size should be <= use[yr, vid]
        relevant_alloc = lpSum(alloc[(yr, s, d, vid)] for (s, d) in forecast_demand.keys() if vehicle_props[vid]['Size'] == s)
        prob += (relevant_alloc <= use[(yr, vid)], f"Allocation_use_{yr}_{vid}")

# Constraint 7: Carbon emissions constraint
# (A detailed emission calculation using fuel consumption, distance, and fuel emission factors should be added.)
for yr in years:
    # For each vehicle allocation, calculate emission: sum over fuel types if needed.
    # Here we add a placeholder constraint ensuring total emission cost is below the limit.
    total_emission = lpSum(alloc[(yr, s, d, vid)] * 0.2  # placeholder emission per km
                            for (s, d) in forecast_demand.keys() 
                            for vid in vehicles if vehicle_props[vid]['Size'] == s)
    carbon_limit = float(carbon_limits_df.loc[carbon_limits_df['Year'] == yr, 'Carbon emission CO2/kg'])
    prob += (total_emission <= carbon_limit, f"Carbon_limit_{yr}")

# [Additional constraints]
# • Insurance, maintenance and depreciation calculations per vehicle lifetime
# • Constraint that vehicles must be sold within 10 years of purchase
# • Fuel type matching with vehicle (ensuring correct fuel usage)
# • Demand segmentation per vehicle size and distance buckets
# You must similarly add constraints (using your provided percentages and rules) to capture all details.

# ====================
# 5. Solve the Model
# ====================

print("Starting optimization...")
prob.solve()
print("Optimization complete. Status:", prob.status)

# ====================
# 6. Extract and Save Results
# ====================

results = []
for yr in years:
    for vid in vehicles:
        b = buy.get((yr, vid))
        u = use.get((yr, vid))
        s = sell.get((yr, vid))
        if b is not None and b.varValue and b.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(b.varValue), 'Type': 'Buy', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if u is not None and u.varValue and u.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(u.varValue), 'Type': 'Use', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if s is not None and s.varValue and s.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(s.varValue), 'Type': 'Sell', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
                            
# Also include allocations per demand bucket if needed
alloc_results = []
for (yr, s, d, vid), var in alloc.items():
    if var.varValue and var.varValue > 0:
        alloc_results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(var.varValue),
                              'Type': 'Use', 'Fuel': None, 'Distance': d,
                              'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})

results_df = pd.DataFrame(results)
alloc_df = pd.DataFrame(alloc_results)

# Save results to csv
results_df.to_csv("fleet_operations.csv", index=False)
alloc_df.to_csv("demand_allocation.csv", index=False)

print("Results saved.")


KeyError: "['Demand'] not in index"

In [14]:
import pandas as pd
import numpy as np
from prophet import Prophet
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger

# ====================
# 1. Demand Forecasting
# ====================

# Read demand.csv with headers: Year, Size, Distance, Demand (km)
demand_df = pd.read_csv("data/demand.csv")

# Forecast demand per (Size, Distance) combination using Prophet.
forecast_horizon = list(range(2023, 2039))
forecast_demand = {}  # dict with key: (Size, Distance) -> forecast DataFrame

for (size, distance), group in demand_df.groupby(["Size", "Distance"]):
    # Use 'Year' as date and 'Demand (km)' as the demand value
    df_prophet = group[['Year', 'Demand (km)']].rename(columns={'Year': 'ds', 'Demand (km)': 'y'})
    # Convert year to datetime (using Jan 1 as date)
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'].astype(str) + '-01-01')
    
    model = Prophet(yearly_seasonality=False, daily_seasonality=False)
    model.fit(df_prophet)
    
    # Create dataframe for future predictions from 2023 to 2038
    future = pd.DataFrame({'ds': pd.to_datetime([str(year) + '-01-01' for year in forecast_horizon])})
    forecast = model.predict(future)
    forecast_df = forecast[['ds', 'yhat']].rename(columns={'ds': 'Year'})
    forecast_df['Year'] = forecast_df['Year'].dt.year
    forecast_demand[(size, distance)] = forecast_df

# ====================
# 2. Fleet Optimization Model Setup
# ====================

# Load other datasets
vehicles_df = pd.read_csv("data/vehicles.csv")  # columns: ID, Vehicle, Size, Year, Cost, Yearly_range, Distance
fuels_df = pd.read_csv("data/fuels.csv")        # columns: Fuel, Year, Emissions, Cost, Cost_Uncertainty
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")  # columns: Year, Carbon emission CO2/kg

# Assume vehicle fuel consumption, insurance, maintenance, etc. are pre-processed as needed.
# Define indices and sets
years = list(range(2023, 2039))
vehicles = vehicles_df['ID'].unique()
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Decision variables:
# buy[year, vehicle] = number of vehicles bought in that year for that model (only if model year equals that year)
buy = {(yr, vid): LpVariable(f"buy_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles if vehicle_props[vid]['Year'] == yr}
# use[year, vehicle] = number of vehicles used in that year (should not exceed fleet available)
use = {(yr, vid): LpVariable(f"use_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles}
# sell[year, vehicle] = vehicles sold at end of that year
sell = {(yr, vid): LpVariable(f"sell_{yr}_{vid}", lowBound=0, cat=LpInteger)
        for yr in years for vid in vehicles}

# Allocation: assign vehicles to meet demand for each (Size, Distance) bucket
alloc = {(yr, s, d, vid): LpVariable(f"alloc_{yr}_{s}_{d}_{vid}", lowBound=0, cat=LpInteger)
         for yr in years
         for (s, d) in forecast_demand.keys()
         for vid in vehicles if vehicle_props[vid]['Size'] == s}

# Create the LP problem:
prob = LpProblem("Fleet_Optimization", LpMinimize)

# ===============
# 3. Objective Function
# ===============
purchase_cost = lpSum(buy[(yr, vid)] * vehicle_props[vid]['Cost'] for (yr, vid) in buy)
resale_value = lpSum(sell[(yr, vid)] * 0.3 * vehicle_props[vid]['Cost']
                     for (yr, vid) in sell)
fuel_cost = lpSum(alloc[(yr, s, d, vid)] * 0.5  # placeholder cost per km
                  for (yr, s, d, vid) in alloc)

prob += (purchase_cost + fuel_cost - resale_value), "Total_Fleet_Cost"

# ===============
# 4. Constraints
# ===============

# Constraint 1: Demand satisfaction for each (Size, Distance) bucket and year
for (s, d), df_forecast in forecast_demand.items():
    for yr in years:
        demand_val = df_forecast.loc[df_forecast['Year'] == yr, 'yhat']
        if not demand_val.empty:
            prob += (lpSum(alloc[(yr, s, d, vid)] * vehicle_props[vid]['Yearly_range']
                           for vid in vehicles if vehicle_props[vid]['Size'] == s)
                     >= float(demand_val), f"Demand_{yr}_{s}_{d}")

# Constraint 2: Distance bucket matching
for yr in years:
    for (s, d), _ in forecast_demand.items():
        for vid in vehicles:
            if vehicle_props[vid]['Size'] == s:
                vehicle_bucket = vehicle_props[vid]['Distance']
                bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
                if bucket_order[vehicle_bucket] < bucket_order[d]:
                    prob += (alloc[(yr, s, d, vid)] == 0, f"Distance_{yr}_{s}_{d}_{vid}")

# Constraint 3: Fleet availability and lifetime (10-year life)
# fleet = {}
# for vid in vehicles:
#     for yr in years:
#         active_fleet = lpSum(buy[(y, vid)] for y in years if y <= yr and yr - y < 10) - \
#                        lpSum(sell[(y, vid)] for y in years if y < yr)
#         fleet[(yr, vid)] = active_fleet
#         prob += (use[(yr, vid)] <= active_fleet, f"Fleet_use_{yr}_{vid}")


# Constraint 3: Fleet availability and lifetime (10-year life)
fleet = {}
for vid in vehicles:
    for yr in years:
        active_fleet = lpSum(buy[(y, vid)] for y in years if y <= yr and (y, vid) in buy and yr - y < 10) - \
                       lpSum(sell[(y, vid)] for y in years if y < yr)
        fleet[(yr, vid)] = active_fleet
        prob += (use[(yr, vid)] <= active_fleet, f"Fleet_use_{yr}_{vid}")


# Constraint 4: Selling limits – at most 20% of the fleet can be sold in any year.
for yr in years:
    total_fleet = lpSum(fleet[(yr, vid)] for vid in vehicles)
    total_sell = lpSum(sell[(yr, vid)] for vid in vehicles)
    prob += (total_sell <= 0.20 * total_fleet, f"Sell_limit_{yr}")

# Constraint 5: Vehicle model purchase year constraint
for (yr, vid) in buy:
    if yr != vehicle_props[vid]['Year']:
        prob += (buy[(yr, vid)] == 0, f"Purchase_year_{yr}_{vid}")

# Constraint 6: Linking allocation with usage
for yr in years:
    for vid in vehicles:
        relevant_alloc = lpSum(alloc[(yr, s, d, vid)] for (s, d) in forecast_demand.keys() if vehicle_props[vid]['Size'] == s)
        prob += (relevant_alloc <= use[(yr, vid)], f"Allocation_use_{yr}_{vid}")

# Constraint 7: Carbon emissions constraint (placeholder)
for yr in years:
    total_emission = lpSum(alloc[(yr, s, d, vid)] * 0.2  # placeholder emission per km
                           for (s, d) in forecast_demand.keys() 
                           for vid in vehicles if vehicle_props[vid]['Size'] == s)
    carbon_limit = float(carbon_limits_df.loc[carbon_limits_df['Year'] == yr, 'Carbon emission CO2/kg'])
    prob += (total_emission <= carbon_limit, f"Carbon_limit_{yr}")

# [Additional constraints would go here.]

# ====================
# 5. Solve the Model
# ====================

print("Starting optimization...")
prob.solve()
print("Optimization complete. Status:", prob.status)

# ====================
# 6. Extract and Save Results
# ====================

results = []
for yr in years:
    for vid in vehicles:
        b = buy.get((yr, vid))
        u = use.get((yr, vid))
        s = sell.get((yr, vid))
        if b is not None and b.varValue and b.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(b.varValue), 'Type': 'Buy', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if u is not None and u.varValue and u.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(u.varValue), 'Type': 'Use', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if s is not None and s.varValue and s.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(s.varValue), 'Type': 'Sell', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})

alloc_results = []
for (yr, s, d, vid), var in alloc.items():
    if var.varValue and var.varValue > 0:
        alloc_results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(var.varValue),
                              'Type': 'Use', 'Fuel': None, 'Distance': d,
                              'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})

results_df = pd.DataFrame(results)
alloc_df = pd.DataFrame(alloc_results)

results_df.to_csv("fleet_operations.csv", index=False)
alloc_df.to_csv("demand_allocation.csv", index=False)

print("Results saved.")


11:28:12 - cmdstanpy - INFO - Chain [1] start processing
11:28:13 - cmdstanpy - INFO - Chain [1] done processing
11:28:13 - cmdstanpy - INFO - Chain [1] start processing
11:28:13 - cmdstanpy - INFO - Chain [1] done processing
11:28:13 - cmdstanpy - INFO - Chain [1] start processing
11:28:13 - cmdstanpy - INFO - Chain [1] done processing
11:28:13 - cmdstanpy - INFO - Chain [1] start processing
11:28:13 - cmdstanpy - INFO - Chain [1] done processing
11:28:14 - cmdstanpy - INFO - Chain [1] start processing
11:28:14 - cmdstanpy - INFO - Chain [1] done processing
11:28:14 - cmdstanpy - INFO - Chain [1] start processing
11:28:14 - cmdstanpy - INFO - Chain [1] done processing
11:28:14 - cmdstanpy - INFO - Chain [1] start processing
11:28:14 - cmdstanpy - INFO - Chain [1] done processing
11:28:14 - cmdstanpy - INFO - Chain [1] start processing
11:28:14 - cmdstanpy - INFO - Chain [1] done processing
11:28:15 - cmdstanpy - INFO - Chain [1] start processing
11:28:15 - cmdstanpy - INFO - Chain [1]

Starting optimization...
Optimization complete. Status: 1
Results saved.


In [2]:
pip install prophet

     ---------------------------------------- 13.3/13.3 MB 5.4 MB/s eta 0:00:00
     -------------------------------------- 820.7/820.7 kB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 94.5/94.5 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install pulp

     ---------------------------------------- 17.7/17.7 MB 5.6 MB/s eta 0:00:00
